# Time Series Project - Store Sales

#### Cosimo Carlo Canova

## Objective :

### To predict sales of different products by store and household based on historical data, including promotional factors.

## Tasks :

###  - Analyze the dataset features: store_nbr, family, onpromotion, sales (target), date
### - Identify any relationships between variables, such as the impact of promotions on sales or seasonal differences between stores and products.

## Overview

|Time|Content|
|---|---|
|1|Importing and Loading Data|
|2|Data Cleaning and Pre-processing|
|3|Exploratory Data Analysis (EDA)|
|4|Data Preparation for the Model| 
|5|Development of the Time Series Model|
|6|Model Evaluation|
|7|Model Optimization|
|8|Implementation and Reporting|
|9|Monitoring the Model in Production| 
|10|Internal Sensitivity Analysis|
|11|Development of a Monitoring Dashboard|
|12|Cross-Validation| 
|13|Residual Analysis|
|14|Conclusion and Future Work|


# Project

## 1. Importing and Loading Data

Import the necessary libraries:

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [53]:
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

Load datasets:

In [4]:
train_data = pd.read_csv('data/store-sales-time-series-forecasting/train.csv')

Review the data:

In [ ]:
print(train_data.head())
print(train_data.info())
print(train_data.describe())

In [ ]:
train_data.head()

## 2. Data Cleaning and Pre-processing

### Handling missing data:

Check for missing values:

In [ ]:
train_data.isnull().sum()

### Encoding categorical variables:

If necessary, transform categorical variables such as store_nbr and family into numbers or dummy variables:

In [8]:
train_data = pd.get_dummies(train_data, columns=['family','store_nbr'], drop_first=True)

In [ ]:
train_data.head()

### Creating new features:

Add useful columns such as: Month, day, week.

In [10]:
#train_data['month'] = train_data['date'].dt.month
#train_data['day_of_week'] = train_data['date'].dt.dayofweek
#train_data['week_of_year'] = train_data['date'].dt.isocalendar().week

The error you're encountering indicates that the date column in your train_data DataFrame is not being recognized as a datetime-like object. Here’s how you can address this issue:

Convert the date column to datetime: Make sure the date column is in the correct format by using pd.to_datetime().

In [11]:
train_data['date'] = pd.to_datetime(train_data['date'])

Check for NaT values: After conversion, check if there are any NaT values that might indicate conversion issues.

In [ ]:
print(train_data['date'].isna().sum())

Re-apply the .dt accessor: Once the date column is confirmed as a datetime type, you can re-run your original code:

In [13]:
train_data['month'] = train_data['date'].dt.month
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['week_of_year'] = train_data['date'].dt.isocalendar().week

## Visualize:

In [ ]:
train_data.head()

In [ ]:
train_data.columns

In [ ]:
train_data['date'].unique()



In [ ]:
train_data.nunique()

### Tabular Display

In [ ]:
print(train_data.head())

### Plots with Matplotlib

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_data['date'], train_data['sales'])  # Replace 'sales' with the column you want to visualize
plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

###   Plots with Seaborn

In [ ]:
sns.lineplot(data=train_data, x='date', y='sales')  # Replace 'sales' with the column you want to visualize
plt.title('Sales Over Time')
plt.show()


In [ ]:
pip install --upgrade seaborn pandas

In [ ]:
import seaborn as sns

sns.lineplot(data=train_data, x='date', y='sales')  # Replace 'sales' with the column you want to visualize
plt.title('Sales Over Time')
plt.show()



In [ ]:
import numpy as np

# Check for infinite values
print(np.isinf(train_data).sum())

# Check for NaN values
print(train_data.isna().sum())



### Bar Charts

In [ ]:
monthly_sales = train_data.groupby('month')['sales'].sum().reset_index()
sns.barplot(data=monthly_sales, x='month', y='sales')
plt.title('Monthly Sales')
plt.show()

### Heatmaps: 

If you want to visualize correlations or data distributed in a matrix.

In [ ]:
# Calculate the correlation matrix
correlation_matrix = train_data.corr()

# Set up a larger figure size
plt.figure(figsize=(80, 60))  # Adjust width and height as needed

# Create the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

# Add a title
plt.title('Correlation Matrix')

# Show the plot
plt.show()


In [26]:
# Save the heatmap as a PDF file
# plt.savefig('correlation_matrix.pdf', bbox_inches='tight')  # Save as a PDF

## 3. Exploratory Data Analysis (EDA)

Objectives: Understand trends, seasonality, and correlations in the data.

Steps:

### Sales distribution: Analyze the distribution of sales globally and by store:

In [27]:
# pip install --upgrade seaborn pandas


In [ ]:

# Set a larger figure size
plt.figure(figsize=(10, 6))

# Improved histogram plot bwith labels, title, and style
sns.histplot(train_data['sales'], bins=50, kde=True, color='skyblue')

# Add labels and title
plt.title('Sales Distribution', fontsize=16)
plt.xlabel('Sales', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Add gridlines
plt.grid(True)

# Show the plot
plt.show()


sns.histplot(): Plots a histogram to show the frequency distribution of sales.

bins=50: Specifies the number of bars in the histogram, providing more granularity.

kde=True: Adds a smoothed line to estimate the probability density function, which gives a visual sense of the distribution's shape.

Key Insights:

Check for skewness: If sales are heavily skewed, transformations like log scaling might be necessary.

Identify if most sales cluster around a certain range or if there are multiple modes (peaks).

Look for outliers that may need special attention (e.g., unusually high or low sales).

### Trends Over Time

Why this is important:

Analyzing sales over time helps you identify:

Seasonality (e.g., whether sales peak at certain times of the year).

Trends (e.g., increasing or decreasing sales over time).

Anomalies (e.g., sudden spikes or drops that may indicate events like promotions or holidays).

What to do:

Visualize how total sales evolve over time using a time series plot. Aggregating sales by day, week, or month will help detect long-term patterns.

How to do it:


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the Seaborn style
sns.set(style="whitegrid")

# Create a figure and axis
plt.figure(figsize=(12, 6))

# Group by date and plot total sales over time
train_data.groupby('date')['sales'].sum().plot(color='royalblue', linewidth=2)

# Add title and labels with better font sizes
plt.title('Total Sales Over Time', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Total Sales', fontsize=14)

# Rotate x-axis labels for better readability and format as dates
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Add gridlines for y-axis
plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)

# Show the plot
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
train_data.groupby('date')['sales'].sum().plot()
plt.title('Sales Over Time')
plt.show()


### Impact of Promotions on Sales

Why this is important:

Promotions often have a significant impact on sales.

Understanding the relationship between promotions and sales helps you determine how much weight you should give to promotional features in the model. 

This can lead to better feature engineering or the design of targeted promotional campaigns.

What to do:

You will use a boxplot to compare sales distributions for items that were on promotion vs. those that were not.


In [ ]:
# Create a boxplot to compare sales with and without promotions
sns.boxplot(data=train_data, x='onpromotion', y='sales')

# Add title and labels
plt.title('Impact of Promotions on Sales')
plt.xlabel('On Promotion (1=Yes, 0=no)')
plt.ylabel('Sales')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set a larger figure size and Seaborn style
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")

# Create a boxplot with a more appealing color palette
sns.boxplot(data=train_data, x='onpromotion', y='sales', palette='coolwarm')

# Add title and labels with better font sizes
plt.title('Impact of Promotions on Sales', fontsize=16)
plt.xlabel('On Promotion (1 = Yes, 0 = No)', fontsize=14)
plt.ylabel('Sales', fontsize=14)

# Add gridlines for y-axis for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)

# Show the plot
plt.tight_layout()  # Adjust layout for better fitting
plt.show()


In [ ]:
sns.boxplot(data=train_data, x='onpromotion', y='sales')
plt.show()


### Additional Steps to Enhance EDA:

Correlations Between Features:

You can use a heatmap to explore correlations between different numerical features (e.g., sales, onpromotion, store number).

This can help identify whether certain stores or product categories have stronger responses to promotions.

In [ ]:
corr = train_data.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
print(train_data.dtypes)

In [ ]:
numeric_data = train_data.select_dtypes(include=[np.number])
corr = numeric_data.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

Handle Non-Numeric Data: 

If there are categorical features (e.g., store_nbr, family, etc.) that you want to include in the analysis, consider encoding them before computing the correlation matrix. 

For example, you could use one-hot encoding:

In [37]:
# train_data_encoded = pd.get_dummies(train_data, columns=['store_nbr', 'family...'], drop_first=True)
# numeric_encoded_data = train_data_encoded.select_dtypes(include=[np.number])
# corr = numeric_encoded_data.corr()
# sns.heatmap(corr, annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix with Encoded Features')
# plt.show()


## 4. Data Preparation for the Model

### Sort data by date:

In [38]:
train_data = train_data.sort_values(by=['date'])

In [ ]:
# Histogram. A histogram is useful for understanding the distribution of numerical data.

# Example: Visualizing a column named 'sales'
plt.figure(figsize=(10, 6))
plt.hist(train_data['sales'], bins=30, color='skyblue', edgecolor='black')
plt.title('Sales Distribution')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.grid(axis='y')
plt.show()


In [ ]:
# Line Plot. A line plot can show trends over time, especially if the column is indexed by date.

plt.figure(figsize=(12, 6))
plt.plot(train_data['date'], train_data['sales'], color='royalblue', linewidth=2)
plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
# Box Plot. A box plot is helpful for visualizing the distribution and identifying outliers.

plt.figure(figsize=(10, 6))
sns.boxplot(y=train_data['sales'], color='lightgreen')
plt.title('Box Plot of Sales')
plt.ylabel('Sales')
plt.grid(axis='y')
plt.show()


In [ ]:
# Violin Plot. A violin plot combines a box plot with a density plot to show the distribution of the data

plt.figure(figsize=(10, 6))
sns.violinplot(y=train_data['sales'], color='lightcoral')
plt.title('Violin Plot of Sales')
plt.ylabel('Sales')
plt.grid(axis='y')
plt.show()


### Split the dataset

In [43]:
# Ensure 'date' is in datetime format
train_data['date'] = pd.to_datetime(train_data['date'])

# Sort the data by date (if not already sorted)
train_data = train_data.sort_values('date')

# Determine the total number of rows
total_rows = len(train_data)

# Calculate the number of rows for the training set (80%)
train_size = int(total_rows * 0.8)

In [ ]:
total_rows, train_size, train_data

In [ ]:
# Get the split date
split_date = train_data['date'].iloc[train_size]

# Now you can use this split date to create your train and test sets
train_set = train_data[train_data['date'] < split_date]
test_set = train_data[train_data['date'] >= split_date]

print(f'Split Date: {split_date}')


In [46]:
#train_set = train_data[train_data['date'] < '2017-01-01']
#test_set = train_data[train_data['date'] >= '2017-01-01']

## 5. Development of the Time Series Model

This section outlines different modeling approaches for forecasting sales based on historical data.

Traditional Time Series Models

ARIMA (AutoRegressive Integrated Moving Average)
Overview: Suitable for datasets with trends and seasonality.

Steps:
- Identify parameters p,d and q using ACF and PACF plots 
- Fit the model

### 1. Identify Parameters p,d and q Using ACF and PACF Plots

Definitions:

p: The number of lag observations included in the model (AR term).

d: The degree of differencing (to make the series stationary).

q: The size of the moving average window (MA term).

Steps to Identify Parameters:
Check for Stationarity:
Before identifying p, d, and q, it’s essential to ensure the time series is stationary. Use the Augmented Dickey-Fuller (ADF) test to check for stationarity

Differencing:
If the series is not stationary (p-value > 0.05), apply differencing to remove trends and seasonality. This gives you d:

#### Check for STATIONARITY:

Differencing

Plot ACF and PACF:

Interpretation:

ACF Plot:
Look for where the ACF plot crosses the significance level (usually the dashed lines) and where it cuts off. This indicates the value of q.

PACF Plot:
Similarly, observe the PACF plot to find where it cuts off to determine p.


In [47]:
# Run all'infinito!

#result = adfuller(train_data['sales'])
#print('ADF statistic:', result[0])
#print('p-value:', result[1])

In [ ]:
# Check for missing values
print(train_data['sales'].isnull().sum())

# Check the type of 'sales' column
print(train_data['sales'].dtype)

In [49]:
# Trasformato la colonna sales in numeri 
train_data['sales'] = pd.to_numeric(train_data['sales'], errors='coerce')

Explanation:
Handling Missing/Non-Numeric Data: The adfuller function requires a complete numeric series. Missing or non-numeric values will lead to issues, potentially causing the test to run indefinitely.

Reducing Dataset Size: If the dataset is large, reducing the size helps test if the function works on a smaller set and gives you feedback on where the issue lies.

In [51]:
# Nonostante ciò non funziona, quindi nella casella successiva riduco le dimensioni del data set così da rendere il calcolo più veloce.

# Running ADF Test on the cleaned 'sales' column
#result = adfuller(train_data['sales'])
#print('ADF Statistic:', result[0])
#print('p-value:', result[1])

In [ ]:
sample_data = train_data.sample(frac=0.1, random_state=42)  # Using 10% of the data
result = adfuller(sample_data['sales'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

Interpretation:
Stationarity: A p-value of 0.0 means the null hypothesis of the ADF test (which assumes that the series is non-stationary) can be rejected with very high confidence. In other words, the sales series is already stationary.

Next Step: Since your data is stationary, there is no need to apply differencing (d = 0). Differencing is only necessary when the series is non-stationary, indicated by a p-value > 0.05.

#### Plot ACF and PACF

Now, you can proceed with identifying the other parameters for your ARIMA model—p (autocorrelation lag) and q (moving average)—using ACF and PACF plots, since the d parameter is set to 0.


In [ ]:
plt.figure(figsize=(12, 6))

# ACF (for q parameter)
plt.subplot(121)
plot_acf(sample_data['sales'], lags=40, ax=plt.gca())
plt.title('Autocorrelation Function (ACF)')

# PACF (for p parameter)
plt.subplot(122)
plot_pacf(sample_data['sales'], lags=40, ax=plt.gca())
plt.title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

Interpretation:

ACF Plot: Since there is no significant autocorrelation at any lag greater than 0, it indicates that the MA (Moving Average) component, denoted as q, should be 0.

PACF Plot: Similarly, the PACF also shows no significant partial autocorrelation beyond lag 0, suggesting that the AR (AutoRegressive) component, denoted as p, should also be 0.

What does this mean?

Both the ACF and PACF plots indicate that there’s no strong autocorrelation in the data, and the model you are working with is likely not benefiting from AR or MA terms. 

Essentially, this points towards a simple ARIMA(0, d, 0) model, where d is the order of differencing that has been applied or needs to be applied.

### Fit the model

In [ ]:
 # Define the ARIMA model
model = ARIMA(train_data['sales'], order=(0, 2, 0))  # Here, p and q are determined from the plots
model_fit = model.fit()

# Summary of the model
print(model_fit.summary())

## 6. Model Evaluation

## 7. Model Optimization

## 8. Implementation and Reporting

## 9. Monitoring the Model in Production

## 10. Internal Sensitivity Analysis

## 11. Development of a Monitoring Dashboard

## 12. Cross-Validation

## 13. Residual Analysis

## 14. Conclusion and Future Work